In [ ]:
import subprocess
import json
import os
from constants import *
from dotenv import load_dotenv
from web3 import Web3, IPCProvider
from eth_account import Account
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from getpass import getpass
from web3.auto.gethdev import w3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
load_dotenv()

In [ ]:
def derive_wallets(mnemonic, coin, numderive):
    mnemonic = os.getenv("mnemonic")
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --coin = "{coins}" --numderive = "{numderive}" --format=json'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys

In [ ]:
derive_wallets(mnemonic, ETH, 2)

In [ ]:
def coins():
    coin_dict = {
        'btc-test' : derive_wallets(mnemonic, BTCTEST, 3),
        'eth' : derive_wallets(mnemonic, ETH, 3)
        #add others
    }

In [ ]:
coins()

In [ ]:
coins()[BTCTEST][0]['privkey']

In [ ]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    else:
        r

In [ ]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

private_key = os.getenv("PRIVATE_KEY")

account_one = Account.from_key(private_key)

In [ ]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }

In [ ]:
def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [ ]:
key = wif_to_key("")

# replace with different addresses
addresses = [
    "0xc3879B456DAA348a16B6524CBC558d2CC984722c",
    "0xA29f7E79ECEA4cE30DD78cfeb9605D9aFf5143a5",
]

outputs = []

for address in addresses:
    outputs.append((address, 0.001, "btc"))

print(key.send(outputs))